In [1]:
import cv2
import numpy as np

def initialize_video_capture(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError("Error opening video file")
    return cap

def get_video_properties(cap):
    return (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),
            int(cap.get(cv2.CAP_PROP_FPS)))

def initialize_video_writer(output_path, fourcc, fps, width, height):
    return cv2.VideoWriter(output_path, fourcc, fps, (width, height))

def save_logs(logs_path, data):
    with open(logs_path, "a") as f:
        # Extraemos los datos del diccionario y los guardamos en el archivo
        for key, value in data.items():
            f.write(f"{key}: {value}\n")

def detect_shoplifting(frame, model, confidence_threshold=0.8):
    """
    Detecta si una persona está robando en el frame utilizando el modelo de detección de robo.
    
    Args:
        frame (numpy.ndarray): El frame de video.
        model (YOLO): El modelo YOLO para detección de robo.
        confidence_threshold (float): Umbral de confianza para considerar una detección válida.
    
    Returns:
        numpy.ndarray: El frame con las anotaciones de detección de robo.
    """

    # Colores para diferenciar a las personas
    ROBBERY_COLOR = (0, 0, 255)  # Rojo para personas robando
    NORMAL_COLOR = (0, 255, 0)    # Verde para personas normales

    # Definir estados de detección
    shoplifting_status = "Robando"
    not_shoplifting_status = "No robando"
    result = model.predict(frame)
    cc_data = np.array(result[0].boxes.data)

    if len(cc_data) != 0:
        xywh = np.array(result[0].boxes.xywh).astype("int32")
        xyxy = np.array(result[0].boxes.xyxy).astype("int32")
        
        for (x1, y1, x2, y2), (_, _, _, _), (_, _, _, _, conf, clas) in zip(xyxy, xywh, cc_data):
            if conf >= confidence_threshold:  # Solo considerar detecciones con alta confianza
                if clas == 1:  # Clase 1: Robo
                    color = ROBBERY_COLOR  # Rojo para personas robando
                    status = shoplifting_status
                else:  # Clase 0: No robo
                    color = NORMAL_COLOR  # Verde para personas normales
                    status = not_shoplifting_status

                # Dibujar el cuadro alrededor de la persona
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

                # Mostrar la confianza como texto
                text = f"{status} {conf * 100:.2f}%"
                cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    return frame

In [ ]:
import cv2
import numpy as np
import time
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator
from datetime import datetime

video_path = "../tests/mall_sim.mp4"
model_path = "../src/models/yolo11x.pt"
output_path = "../outputs/mall_sim.mp4"

In [24]:
class ZoneManager:
    def __init__(self):
        self.zones = {}
        self.hourly_traffic = {}
        self.crossing_events = [] 
        
    def record_traffic_metrics(self, event):
        """Record detailed traffic metrics for each zone"""
        zone_id = event["zone_id"]
        timestamp = event["timestamp"]
        
        # Convert timestamp to datetime for time-based analysis
        dt = datetime.fromtimestamp(timestamp)
        hour_key = dt.strftime("%Y-%m-%d %H:00")
        day_key = dt.strftime("%Y-%m-%d")
        
        # Initialize structure if needed
        if zone_id not in self.hourly_traffic:
            self.hourly_traffic[zone_id] = {}
        if zone_id not in self.daily_traffic:
            self.daily_traffic[zone_id] = {}
            
        # Record entry/exit by hour
        if hour_key not in self.hourly_traffic[zone_id]:
            self.hourly_traffic[zone_id][hour_key] = {"entries": 0, "exits": 0}
        
        # Record entry/exit by day
        if day_key not in self.daily_traffic[zone_id]:
            self.daily_traffic[zone_id][day_key] = {"entries": 0, "exits": 0}
        
        # Update counts
        if event["event_type"] == "entry":
            self.hourly_traffic[zone_id][hour_key]["entries"] += 1
            self.daily_traffic[zone_id][day_key]["entries"] += 1
        else:  # exit
            self.hourly_traffic[zone_id][hour_key]["exits"] += 1
            self.daily_traffic[zone_id][day_key]["exits"] += 1

    def add_zone(self, zone_id, zone_type, position, entry_direction, exit_direction, color=(0, 255, 0)):
        """
        Add a new zone or line.
        
        Args:
            zone_id (str): Unique identifier for the zone
            zone_type (str): "line" (horizontal/vertical) or "area" (polygon)
            position: For lines: (line_type, position), For areas: list of points
            entry_direction (str): Direction for entry (can be custom text)
            exit_direction (str): Direction for exit (can be custom text)
            color (tuple): RGB color for visualization
        """
        self.zones[zone_id] = {
            "type": zone_type,
            "position": position,
            "entry_direction": entry_direction,
            "exit_direction": exit_direction,
            "color": color,
            "entry_times": {},  # Track entry times for each person in this zone
            "time_spent_list": []  # List to store time spent in this zone
        }
        
        if zone_type == "line":
            line_type, line_position = position
            self.zones[zone_id]["line_start"], self.zones[zone_id]["line_end"], self.zones[zone_id]["crossed_line"] = \
                self._configure_line(line_type, line_position, entry_direction, exit_direction)
        
    def _configure_line(self, line_type, line_position, entry_direction, exit_direction):
        """Configure a line and its crossing conditions"""
        if line_type == "horizontal":
            line_start = (0, line_position)
            line_end = (self.frame_width, line_position)
        else:  # Vertical
            line_start = (line_position, 0)
            line_end = (line_position, self.frame_height)
            
        # Define conditions for crossing based on direction
        def crossed_line(current_pos, last_pos, line_pos, direction):
            if line_type == "horizontal":
                last_y, current_y = last_pos[1], current_pos[1]
                if direction == "down":
                    return last_y < line_pos and current_y >= line_pos
                elif direction == "up":
                    return last_y > line_pos and current_y <= line_pos
            else:  # Vertical
                last_x, current_x = last_pos[0], current_pos[0]
                if direction == "right":
                    return last_x < line_pos and current_x >= line_pos
                elif direction == "left":
                    return last_x > line_pos and current_x <= line_pos
            return False
            
        return line_start, line_end, crossed_line
        
    def set_frame_dimensions(self, width, height):
        """Set the frame dimensions for calculations"""
        self.frame_width = width
        self.frame_height = height
        """Draw all zones/lines on the frame"""
        for zone_id, zone in self.zones.items():
            if zone["type"] == "line":
                cv2.line(frame, zone["line_start"], zone["line_end"], zone["color"], 2)
                # Add text label near the line
                text_pos = (zone["line_start"][0] + 10, zone["line_start"][1] - 10)
                cv2.putText(frame, f"Zone {zone_id}", text_pos, 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, zone["color"], 2)
            elif zone["type"] == "area":
                # Draw polygon areas
                points = np.array(zone["position"], np.int32).reshape((-1, 1, 2))
                cv2.polylines(frame, [points], True, zone["color"], 2)
                
                # Add some transparency to the polygon
                overlay = frame.copy()
                cv2.fillPoly(overlay, [points], (*zone["color"], 128))  # Semi-transparent color
                cv2.addWeighted(overlay, 0.3, frame, 0.7, 0, frame)
                
                # Add label for the area
                centroid_x = sum(p[0] for p in zone["position"]) // len(zone["position"])
                centroid_y = sum(p[1] for p in zone["position"]) // len(zone["position"])
                cv2.putText(frame, f"Zone {zone_id}", (centroid_x, centroid_y), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, zone["color"], 2)
                
        return frame
        
    def define_zones_interactive(first_frame):
        """Allow user to draw zones on the first frame and set entry/exit directions"""
        zones = {}
        temp_points = []
        current_zone_id = ""
        
        def mouse_callback(event, x, y, flags, param):
            nonlocal temp_points, current_zone_id
            
            if event == cv2.EVENT_LBUTTONDOWN:
                temp_points.append((x, y))
                cv2.circle(display_frame, (x, y), 5, (0, 0, 255), -1)
                # Draw lines to connect points
                if len(temp_points) > 1:
                    cv2.line(display_frame, temp_points[-2], temp_points[-1], (255, 0, 0), 2)
                cv2.imshow("Define Zones", display_frame)
                
        display_frame = first_frame.copy()
        cv2.namedWindow("Define Zones")
        cv2.setMouseCallback("Define Zones", mouse_callback)
        
        print("=== Zone Definition Mode ===")
        print("1. Click points to define a zone boundary")
        print("2. Press 'c' to complete the current zone")
        print("3. After completing a zone, follow prompts to set entry/exit parameters")
        print("4. Press 'r' to reset the current zone")
        print("5. Press 'q' when finished defining all zones")
        
        zone_count = 0
        
        while True:
            cv2.imshow("Define Zones", display_frame)
            key = cv2.waitKey(1) & 0xFF
            
            if key == ord('c') and temp_points:
                # Complete the zone by drawing the closing line
                if len(temp_points) > 2:
                    cv2.line(display_frame, temp_points[-1], temp_points[0], (255, 0, 0), 2)
                
                # Create a filled polygon with transparency for better visualization
                overlay = display_frame.copy()
                zone_count += 1
                zone_id = f"zone_{zone_count}"
                
                # Draw the completed zone
                points = np.array(temp_points, np.int32).reshape((-1, 1, 2))
                cv2.fillPoly(overlay, [points], (0, 255, 0, 128))  # Semi-transparent green
                cv2.addWeighted(overlay, 0.4, display_frame, 0.6, 0, display_frame)
                
                # Add text label in the center of the polygon
                center_x = sum(p[0] for p in temp_points) // len(temp_points)
                center_y = sum(p[1] for p in temp_points) // len(temp_points)
                cv2.putText(display_frame, f"Zone {zone_count}", 
                        (center_x, center_y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                
                # Get direction settings for this zone
                print(f"\nZone {zone_count} defined. Now set entry/exit parameters:")
                print("For polygon zones, you're defining what counts as entry and exit.")
                entry_direction = input("Enter entry label (e.g., 'from north', 'from entrance'): ")
                exit_direction = input("Enter exit label (e.g., 'to south', 'to exit'): ")
                
                # Define zone color - can add more options here
                color_choice = input("Choose zone color (r=red, g=green, b=blue, m=magenta, default=green): ").lower()
                if color_choice == 'r':
                    color = (0, 0, 255)  # Red in BGR
                elif color_choice == 'b':
                    color = (255, 0, 0)  # Blue in BGR
                elif color_choice == 'm':
                    color = (255, 0, 255)  # Magenta in BGR
                else:
                    color = (0, 255, 0)  # Green in BGR
                
                # Store the zone
                zones[zone_id] = {
                    "type": "area",
                    "points": temp_points.copy(),
                    "entry_direction": entry_direction,
                    "exit_direction": exit_direction,
                    "color": color
                }
                
                temp_points = []
                print(f"Zone {zone_count} saved. Define next zone or press 'q' to finish.")
                    
            elif key == ord('r'):
                # Reset current zone
                temp_points = []
                display_frame = first_frame.copy()
                # Redraw existing zones
                for z_id, z_data in zones.items():
                    pts = np.array(z_data["points"], np.int32).reshape((-1, 1, 2))
                    cv2.polylines(display_frame, [pts], True, z_data["color"], 2)
                    
                    # Add text label in the center of the polygon
                    center_x = sum(p[0] for p in z_data["points"]) // len(z_data["points"])
                    center_y = sum(p[1] for p in z_data["points"]) // len(z_data["points"])
                    cv2.putText(display_frame, z_id, 
                            (center_x, center_y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                
            elif key == ord('q'):
                break
        
        cv2.destroyWindow("Define Zones")
        return zones
    
    def get_advanced_stats(self):
        """Get advanced statistics for all zones"""
        stats = {}
        current_time = time.time()
        
        for zone_id, zone in self.zones.items():
            # Basic stats
            people_currently_in_zone = len(zone["entry_times"])
            
            # Average dwell time (from completed visits)
            avg_dwell = sum(zone["time_spent_list"]) / len(zone["time_spent_list"]) if zone["time_spent_list"] else 0
            
            # Current dwell times of people still in the zone
            current_dwell_times = [current_time - entry_time for entry_time in zone["entry_times"].values()]
            
            # Maximum occupancy recorded
            max_occupancy = getattr(zone, "max_occupancy", people_currently_in_zone)
            if people_currently_in_zone > max_occupancy:
                zone["max_occupancy"] = people_currently_in_zone
                max_occupancy = people_currently_in_zone
            
            # Calculate occupancy rate (current occupancy / max recorded)
            occupancy_rate = people_currently_in_zone / max_occupancy if max_occupancy > 0 else 0
            
            stats[zone_id] = {
                "current_occupancy": people_currently_in_zone,
                "max_occupancy": max_occupancy,
                "occupancy_rate": occupancy_rate,
                "average_dwell_time": avg_dwell,
                "current_dwell_times": current_dwell_times,
                "longest_current_visit": max(current_dwell_times) if current_dwell_times else 0
            }
            
        return stats

    def check_crossings(self, track_id, current_pos, last_pos):
        """Check if a person has crossed any zones/lines"""
        events = []  # List to store triggered events
        
        for zone_id, zone in self.zones.items():
            if zone["type"] == "line":
                # Line crossing logic
                line_position = zone["position"][1]
                
                # Check for entry
                if zone["crossed_line"](current_pos, last_pos, line_position, zone["entry_direction"]):
                    if track_id not in zone["entry_times"]:
                        zone["entry_times"][track_id] = time.time()  # Record entry time
                        event_data = {
                            "zone_id": zone_id,
                            "event_type": "entry",
                            "track_id": track_id,
                            "direction": zone["entry_direction"],
                            "timestamp": zone["entry_times"][track_id]
                        }
                        events.append(event_data)
                
                # Check for exit
                if zone["crossed_line"](current_pos, last_pos, line_position, zone["exit_direction"]):
                    if track_id in zone["entry_times"]:
                        exit_time = time.time()
                        time_spent = exit_time - zone["entry_times"][track_id]
                        zone["time_spent_list"].append(time_spent)
                        event_data = {
                            "zone_id": zone_id,
                            "event_type": "exit",
                            "track_id": track_id,
                            "direction": zone["exit_direction"],
                            "time_spent": time_spent,
                            "timestamp": exit_time
                        }
                        events.append(event_data)
                        del zone["entry_times"][track_id]  # Remove from entry times
                        
            elif zone["type"] == "area":
                # Check for entry into or exit from polygon area
                was_inside = self.point_in_polygon(last_pos, zone["position"])
                is_inside = self.point_in_polygon(current_pos, zone["position"])
                
                # Entry event
                if is_inside and not was_inside:
                    zone["entry_times"][track_id] = time.time()
                    event_data = {
                        "zone_id": zone_id,
                        "event_type": "entry",
                        "track_id": track_id,
                        "direction": zone["entry_direction"],
                        "timestamp": zone["entry_times"][track_id]
                    }
                    events.append(event_data)
                    
                # Exit event
                elif was_inside and not is_inside:
                    if track_id in zone["entry_times"]:
                        exit_time = time.time()
                        time_spent = exit_time - zone["entry_times"][track_id]
                        zone["time_spent_list"].append(time_spent)
                        event_data = {
                            "zone_id": zone_id,
                            "event_type": "exit",
                            "track_id": track_id,
                            "direction": zone["exit_direction"],
                            "time_spent": time_spent,
                            "timestamp": exit_time
                        }
                        events.append(event_data)
                        del zone["entry_times"][track_id]
                    
        return events

    def draw_zones(self, frame):
        """Draw all zones/lines on the frame"""
        for zone_id, zone in self.zones.items():
            if zone["type"] == "line":
                cv2.line(frame, zone["line_start"], zone["line_end"], zone["color"], 2)
                text_pos = (zone["line_start"][0] + 10, zone["line_start"][1] - 10)
                cv2.putText(frame, f"Zone {zone_id}", text_pos, 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, zone["color"], 2)
            elif zone["type"] == "area":
                # Draw polygon areas
                points = np.array(zone["position"], np.int32).reshape((-1, 1, 2))
                cv2.polylines(frame, [points], True, zone["color"], 2)
                
                # Add some transparency to the polygon
                overlay = frame.copy()
                cv2.fillPoly(overlay, [points], (*zone["color"], 128))  # Semi-transparent color
                cv2.addWeighted(overlay, 0.3, frame, 0.7, 0, frame)
                
                # Add label for the area
                centroid_x = sum(p[0] for p in zone["position"]) // len(zone["position"])
                centroid_y = sum(p[1] for p in zone["position"]) // len(zone["position"])
                cv2.putText(frame, f"Zone {zone_id}", (centroid_x, centroid_y), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, zone["color"], 2)
                
        return frame
        
    def get_stats(self):
        """Get simple statistics for all zones"""
        stats = {}
        for zone_id, zone in self.zones.items():
            stats[zone_id] = {
                "people_in_zone": len(zone["entry_times"]),
                "average_time_spent": sum(zone["time_spent_list"]) / len(zone["time_spent_list"]) 
                                     if zone["time_spent_list"] else 0
            }
        return stats
    
    def point_in_polygon(self, point, polygon):
        """
        Check if a point is inside a polygon using ray casting algorithm.
        Args:
            point: (x, y) tuple
            polygon: List of (x, y) tuples forming the polygon
        Returns:
            bool: True if point is inside the polygon
        """
        x, y = point
        n = len(polygon)
        inside = False
        
        p1x, p1y = polygon[0]
        for i in range(1, n + 1):
            p2x, p2y = polygon[i % n]
            if y > min(p1y, p2y):
                if y <= max(p1y, p2y):
                    if x <= max(p1x, p2x):
                        if p1y != p2y:
                            xinters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                        if p1x == p2x or x <= xinters:
                            inside = not inside
            p1x, p1y = p2x, p2y
        
        return inside
        
    def record_crossing_event(self, track_id, zone_id, event_type):
        """Record a crossing event with timestamp"""
        from datetime import datetime
        
        current_time = time.time()
        dt = datetime.fromtimestamp(current_time)
        hour_key = dt.strftime("%Y-%m-%d %H:00")
        
        # Update hourly traffic data
        if zone_id not in self.hourly_traffic:
            self.hourly_traffic[zone_id] = {}
            
        if hour_key not in self.hourly_traffic[zone_id]:
            self.hourly_traffic[zone_id][hour_key] = {"entries": 0, "exits": 0}
            
        if event_type == "entry":
            self.hourly_traffic[zone_id][hour_key]["entries"] += 1
        else:  # exit
            self.hourly_traffic[zone_id][hour_key]["exits"] += 1
        
        # Record the event
        self.crossing_events.append({
            "track_id": track_id,
            "zone_id": zone_id,
            "event_type": event_type,
            "timestamp": current_time,
            "datetime": dt.strftime("%Y-%m-%d %H:%M:%S")
        })
    
    def save_zone_metrics(self, output_path):
        """Save zone metrics to JSON file"""
        import json
        
        # Calculate summary stats for each zone
        summary = {}
        for zone_id in self.zones:
            entries = sum(hour_data["entries"] for hour_data in self.hourly_traffic.get(zone_id, {}).values())
            exits = sum(hour_data["exits"] for hour_data in self.hourly_traffic.get(zone_id, {}).values())
            
            # Find busiest hour
            busiest_hour = None
            max_traffic = 0
            
            for hour, hour_data in self.hourly_traffic.get(zone_id, {}).items():
                hour_traffic = hour_data["entries"] + hour_data["exits"]
                if hour_traffic > max_traffic:
                    max_traffic = hour_traffic
                    busiest_hour = hour
            
            summary[zone_id] = {
                "total_entries": entries,
                "total_exits": exits,
                "busiest_hour": busiest_hour,
                "busiest_hour_traffic": max_traffic if busiest_hour else 0,
                "current_occupancy": len(self.zones[zone_id]["entry_times"]),
                "avg_dwell_time": sum(self.zones[zone_id]["time_spent_list"]) / 
                                 len(self.zones[zone_id]["time_spent_list"]) 
                                 if self.zones[zone_id]["time_spent_list"] else 0
            }
            
        metrics = {
            "zone_summary": summary,
            "hourly_traffic": self.hourly_traffic,
            "crossing_events": self.crossing_events
        }
        
        with open(output_path, 'w') as f:
            json.dump(metrics, f, indent=4)
        
        return metrics

In [25]:
# Before starting the main tracking loop, add this code to define zones interactively
def define_zones():
    # Initialize video capture
    cap = initialize_video_capture(video_path)
    
    # Read the first frame for drawing
    ret, first_frame = cap.read()
    if not ret:
        print("Failed to read video")
        return {}
    
    # Call the interactive zone definition method
    zones = ZoneManager.define_zones_interactive(first_frame)
    
    cap.release()
    return zones

In [28]:
model = YOLO(model_path)
model.to("cuda")
zone_manager = ZoneManager()

cap = initialize_video_capture(video_path)
w, h, fps = get_video_properties(cap)
cap.release()

print("Define custom zones before starting tracking")
custom_zones = define_zones()

zone_manager = ZoneManager()
zone_manager.set_frame_dimensions(w, h)
for zone_id, zone_data in custom_zones.items():
    zone_manager.add_zone(
        zone_id=zone_id,
        zone_type="area",
        position=zone_data["points"],
        entry_direction=zone_data.get("entry_direction", "inside"),
        exit_direction=zone_data.get("exit_direction", "outside"),
        color=zone_data.get("color", (255, 0, 255))
    )

cap = initialize_video_capture(video_path)
out = initialize_video_writer(output_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, w, h)

# Initialize variables
prev_time = time.time()
last_positions = {}
frame_count = 0
last_stats_time = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        print("End of video.")
        break

    frame_count += 1
    annotator = Annotator(frame, line_width=2)
    results = model.track(frame, persist=True)
    frame = zone_manager.draw_zones(frame)

    if results[0].boxes.id is not None:
        boxes = results[0].boxes.xyxy.cpu().numpy()
        track_ids = results[0].boxes.id.int().cpu().tolist()
        class_ids = results[0].boxes.cls.cpu().numpy()

        for box, track_id, class_id in zip(boxes, track_ids, class_ids):
            if int(class_id) == 0:
                centroid_x = int((box[0] + box[2]) / 2)
                centroid_y = int((box[1] + box[3]) / 2)
                current_pos = (centroid_x, centroid_y)
                
                x1, y1, x2, y2 = box.astype(int)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                
                cv2.putText(frame, f"ID: {track_id}", (x1, y1 - 10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                
                cv2.circle(frame, (centroid_x, centroid_y), 4, (0, 0, 255), -1)
                
                if track_id in last_positions:
                    events = zone_manager.check_crossings(track_id, current_pos, last_positions[track_id])
                    for event in events:
                        zone_manager.record_crossing_event(track_id, event["zone_id"], event["event_type"])
                        
                        # Print event information
                        if event["event_type"] == "entry":
                            print(f"Person {event['track_id']} entered zone '{event['zone_id']}'")
                        elif event["event_type"] == "exit":
                            if 'time_spent' in event:
                                print(f"Person {event['track_id']} exited zone '{event['zone_id']}' after {event['time_spent']:.2f} seconds")
                            else:
                                print(f"Person {event['track_id']} exited zone '{event['zone_id']}'")
                
                last_positions[track_id] = current_pos

    current_time = time.time()
    fps = 1 / (current_time - prev_time)
    prev_time = current_time

    # Get statistics for all zones
    stats = zone_manager.get_stats()
    
    # Display overall statistics
    cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    
    # Display zone-specific statistics
    y_pos = 70
    for zone_id, zone_stats in stats.items():
        cv2.putText(frame, f"{zone_id}: {zone_stats['people_in_zone']} people", 
                   (10, y_pos), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        y_pos += 30

    out.write(frame)
    cv2.imshow("object-detection-tracking", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

metrics = zone_manager.save_zone_metrics("../outputs/zone_entry_exit_metrics.json")

out.release()
cap.release()
cv2.destroyAllWindows()

Define custom zones before starting tracking
=== Zone Definition Mode ===
1. Click points to define a zone boundary
2. Press 'c' to complete the current zone
3. After completing a zone, follow prompts to set entry/exit parameters
4. Press 'r' to reset the current zone
5. Press 'q' when finished defining all zones

Zone 1 defined. Now set entry/exit parameters:
For polygon zones, you're defining what counts as entry and exit.
Zone 1 saved. Define next zone or press 'q' to finish.

Zone 2 defined. Now set entry/exit parameters:
For polygon zones, you're defining what counts as entry and exit.
Zone 2 saved. Define next zone or press 'q' to finish.

Zone 3 defined. Now set entry/exit parameters:
For polygon zones, you're defining what counts as entry and exit.
Zone 3 saved. Define next zone or press 'q' to finish.

Zone 4 defined. Now set entry/exit parameters:
For polygon zones, you're defining what counts as entry and exit.
Zone 4 saved. Define next zone or press 'q' to finish.

0: 384x6